In [1]:
import os
import ast
import operator as op
import requests
from urllib.parse import urlencode
from transformers import AutoModelForCausalLM, AutoTokenizer
from transformers.utils import get_json_schema
import json
import re
from typing import Optional
from jinja2 import Template
import torch
import pandas as pd
import numpy as np
from tqdm.auto import tqdm
from sklearn.metrics import mean_absolute_percentage_error

In [2]:
API_NINJAS_KEY = "vC3fWKi6n1nE2D9vnLqxdQ==U6HUomPDVD2qWfl1"
API_NUTRITIONS = 'vC3fWKi6n1nE2D9vnLqxdQ==ilCB2JHp3C6WQGa2'

In [3]:
# define API functions

def get_nutrition(dish: str, debug: False) -> dict:
    try:
        api_url = 'https://api.calorieninjas.com/v1/nutrition?query='
        response = requests.get(api_url + dish, headers={'X-Api-Key': API_NUTRITIONS})
        response.raise_for_status()
        response = response.json()['items']
        if debug:
          print(response)
        return response if response else [{"error": f"No nutrition data for {dish}"}]
    except requests.RequestException as e:
        return [{"error": f"Failed to fetch nutrition: {str(e)}"}]

In [ ]:
# Test API keys
import requests

print("Testing Nutrition API...")
nutrition_resp = requests.get(
    "https://api.calorieninjas.com/v1/nutrition?query=100g pizza",
    headers={'X-Api-Key': API_NUTRITIONS},
    timeout=10
)
print(f"Nutrition API: {nutrition_resp.status_code} - {'OK' if nutrition_resp.status_code == 200 else 'FAIL'}")

if nutrition_resp.status_code == 200:
    print("✓ API working!")

In [4]:
# # define tools json schemas
tools = [
    {
        "name": "get_nutrition",
        "description": "Calculate the total nutrition value (e.g., calories, protein) for a given dish.",
        "parameters": {
            "type": "object",
            "properties": {
                "dish": {
                    "type": "string",
                    "description": "The name of the dish in English, e.g., 'Caesar salad', 'vegetable rice', etc."
                }
            },
            "required": ["dish"]
        }
    }
]

In [5]:
# define utils

def generate_response(model, tokenizer, messages, max_tokens=4096):
    model.eval()
    with torch.inference_mode():
        inputs = tokenizer.apply_chat_template(messages, return_tensors="pt", enable_thinking=False).to(model.device)
        outputs = model.generate(
            inputs,
            max_new_tokens=max_tokens,
            do_sample=False,
            use_cache=True,
            pad_token_id=tokenizer.eos_token_id
        )
    response = tokenizer.decode(outputs[0][inputs.shape[1]:], skip_special_tokens=True)
    return response.strip()

def prepare_messages(
    query: str,
    tools: Optional[list[dict]] = None,
    history: Optional[list[dict[str, str]]] = None
) -> list[dict[str, str]]:
    """Prepare the system and user messages for the given query and tools.

    Args:
        query: The query to be answered.
        tools: The tools available to the user. Defaults to None, in which case an empty list will be passed.
        history: Exchange of messages, including the system_prompt from the first query. Defaults to None.
    """
    if tools is None:
        tools = []
    if history:
        messages = history.copy()
        messages.append({"role": "user", "content": query})
    else:
        messages = [
            {"role": "system", "content": system_prompt.render(tools=json.dumps(tools))},
            {"role": "user", "content": query}
        ]
    return messages

def parse_response(text: str) -> str | list[dict[str, any]]:
    """Parses a response from the model, returning either the parsed list of tool calls or the text response.

    Args:
        text: Response from the model.
    """
    pattern = r"<tool_call>(.*?)</tool_call>"
    matches = re.findall(pattern, text, re.DOTALL)
    tool_calls = []
    if matches:
      for m in matches:
        try:
            parsed = json.loads(m)
            if isinstance(parsed, list):
                tool_calls.extend(parsed)
            else:
                tool_calls.append(parsed)
        except json.JSONDecodeError:
            pass
    return tool_calls

In [6]:
system_prompt = Template(
"""
You are a helpful AI assistant that answers questions about the nutritional content and recipes of various dishes.

# Tools
You have access to a tools called `get_nutrition`. You **must** use this tools whenever a user asks about the nutritional value of a dish or recipe.

# Tools description
get_nutrition(dish: str) -> dict

- **dish**: the name of the dish, written in correct English word order (name first, dish type second).

# IMPORTANT: Donâ€™t call the tool multiple times with the same arguments.
If different arguments didnâ€™t help, return the **best available result** instead of repeating call.

# Output
**ALWAYS return the final answer strictly depending on query type in the format:**
   `<value>` (e.g. 346) for queries about nutrition
   `<yes/no>` (e.g. yes) for queries about presenting ingridients in dish
   `<dish name>` (e.g. Caesar salad) for queries with comparing of two dishes
   `<number of ingridients>` (e.g. 10) for queries about ingridients
   No explanations, no apologies, no additional text.


Use the following format for tool calls:

<tool_call>[
  {"name": "get_nutrition", "arguments": {"dish": "dish_name"}}
]</tool_call>
"""
)

In [7]:
def run_agent(user_query: str, model, tokenizer, system_prompt: Template, debug: bool = False):
    # Initialize messages with tools
    messages = prepare_messages(user_query + ' /no_think', tools=tools)

    used_arguments = []
    # Function calling loop
    while True:
        if debug:
            print(messages[-1])

        response = generate_response(model, tokenizer, messages)
        if debug:
            print(response)
        messages.append({"role": "assistant", "content": response})
        tool_calls = parse_response(response)

        if len(tool_calls) > 0:
            for tool_call in tool_calls:
                function_name = tool_call.get("name")
                arguments = tool_call.get("arguments", {})

                if debug:
                    print('CALL', function_name, arguments)

                if arguments in used_arguments:
                    result = {'error': 'The tool with this arguments already used'}
                elif function_name == "get_nutrition":
                    result = get_nutrition(**arguments, debug=debug)
                    used_arguments.append(arguments)
                else:
                    result = {"error": f"Unknown tool: {function_name}. Please call tool with different name."}

                messages.append({
                    "role": "tool",
                    "content": json.dumps(result),
                })
            continue
        break

    response = response.replace('<think>', '').replace('</think>', '').strip()
    return response

In [8]:
# Load model and tokenizer
model_name = "Qwen/Qwen3-4B"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.float16,
    device_map="auto",
    trust_remote_code=True
)

model.generation_config.temperature = None
model.generation_config.top_p = None
model.generation_config.top_k = None
model.generation_config.do_sample = False

`torch_dtype` is deprecated! Use `dtype` instead!


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [9]:
# PUT test.csv next to the notebook

In [10]:
test_df = pd.read_csv(r'test.csv')
test_df.head()

,id,question
0,0,How many calories are there in 100 gramm of Ca...
1,1,How many ingredients are in lasagna?
2,2,How many apples do I need to cook apple pie?
3,3,How many calories are in 100 gramm of apple pie?
4,4,How much fat is in 100 gramm of grilled chicken?


In [11]:
res_dict = {}
test_data = test_df['question'].to_dict()
for q_id, question in tqdm(test_data.items()):
    answer = run_agent(question, model, tokenizer, system_prompt, debug=False)
    res_dict[q_id] = answer
    print(f"Final answer = {answer}")

res_df = pd.DataFrame({"y_pred": res_dict}).reset_index().rename(columns={'index': 'id'})
res_df.to_csv('submission.csv', index=False)

  0%|          | 0/100 [00:00<?, ?it/s]

The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


Final answer = 160.4
Final answer = 10
Final answer = 1.9
Final answer = 240.1
Final answer = 3.5
Final answer = The protein content in 450 grams of mushroom soup is 6.3 grams.
Final answer = No.
Final answer = 10
Final answer = 152.4
Final answer = 3.4
Final answer = 1
Final answer = yes
Final answer = 11.0
Final answer = 29.4
Final answer = 3.4 g per 100 g. For 260 g, it would be 3.4 * 2.6 = 8.84 g.
Final answer = The nutritional information for blueberry pancakes is provided, but it does not specify the amount of flour required. To determine the number of cups of flour needed, you would need to know the specific recipe or recipe details. Please provide the recipe or additional details about the blueberry pancakes you are referring to.
Final answer = The question is about the amount of Cornstarch needed to cook Mushroom soup, which is a recipe-related query. The tool `get_nutrition` is not suitable for answering this question as it provides nutritional information about a dish, not t

In [ ]:
# Free GPU memory
import gc

del model
del tokenizer
gc.collect()

if torch.cuda.is_available():
    torch.cuda.empty_cache()
    torch.cuda.synchronize()
    print(f"GPU memory freed. Allocated: {torch.cuda.memory_allocated() / 1024**2:.1f} MB")
else:
    print("No CUDA available")